
La parte de la trama de **data** se utiliza para enviar el puerto <GPIO ID> y la parte de **device** para indicar si se desea escribir o leer el puerto indicado 

* device = 0: LEDs 
* device = 1: Switches 

```c
#define def_READ      0
#define def_WRITE     1
```
    
```  
Ingrese un comando: 0R1
[b'\xa3', b'\x00', b'\x00', b'\x01', '0', 'R', '1', b'C']

 Ingrese un comando: 2G0
[b'\xa3', b'\x00', b'\x00', b'\x01', '2', 'G', '0', b'C']```

|    Trama     |      Byte         |     Significado   |    Valor        |
|:------------:|:-----------------:|:------------------:|:--------------:|
| Cabecera     |      Byte 1       |   Init (3b)        |    101         |
| Cabecera     |      Byte 1       |   Tipo  (1b)       |    1/0         |
| Cabecear     |      Byte 1       |   Size (4b)        |  *DataSize*    |
| Cabecera     |      Byte 2       |   Size MSBits      |*Long DataSize* |
| Cabecera     |      Byte 3       |   Size LSBits      |*Long DataSize* |
| Cabecera     |      Byte 4       |   Device           |*READ/WRITE* 1/0|          
| Data         |     *N* Bytes     |   Data             |   *Data*       |
| Fin Trama    |      Byte 1       |   End (3b)         |    010         |
| Fin Trama    |      Byte 1       |   Tipo (1b)        |    1/0         |
| Fin Trama    |      Byte 1       |   Size (4b)        |  *DataSize*    |



 * Hago una AND entre el color y el numero del LED para poder prender ese determinado y implementarlo como un switch case, 
    para no tener que comparar una cadena de caracteres caracter a caracter que lleva mas tiempo 
    
    ```c
    LED_ID = *(frame + FRAME_HEAD_L+1) & *(frame + FRAME_HEAD_L+2);```
    
    
    

La función `DiscreRead` no devuelve lo que tendría que devolver, por lo que se hace una OR de la máscara antes seleccionada con la nueva. Para que se los LEDs ya encendidos no se apaguen. 

Para lograr encender cualquier combinación de LEDs se utilizaron 2 variables de 32 bits para enmascarar el estado de los LEDs y encender o apagar únicamente el indicado por comando. 

```c 
switch(LED_ID){
    case (R0):{	GPO_Mask = GPO_Mask | LED0_R; GPO_SEL = LED0_R; break;}
    case (G0):{	GPO_Mask = GPO_Mask | LED0_G; GPO_SEL = LED0_G; break;}
    case (B0):{	GPO_Mask = GPO_Mask | LED0_B; GPO_SEL = LED0_B; break;}

    case (R1):{	GPO_Mask = GPO_Mask | LED1_R; GPO_SEL = LED1_R; break;}
	case (G1):{	GPO_Mask = GPO_Mask | LED1_G; GPO_SEL = LED1_G; break;}
    case (B1):{	GPO_Mask = GPO_Mask | LED1_B; GPO_SEL = LED1_B; break;}

    case (R2):{	GPO_Mask = GPO_Mask | LED2_R; GPO_SEL = LED2_R; break;}
    case (G2):{	GPO_Mask = GPO_Mask | LED2_G; GPO_SEL = LED2_G; break;}
    case (B2):{	GPO_Mask = GPO_Mask | LED2_B; GPO_SEL = LED2_B; break;}

    case (R3):{	GPO_Mask = GPO_Mask | LED3_R; GPO_SEL = LED3_R; break;}
    case (G3):{	GPO_Mask = GPO_Mask | LED3_G; GPO_SEL = LED3_G; break;}
    case (B3):{	GPO_Mask = GPO_Mask | LED3_B; GPO_SEL = LED3_B; break;}

    default:GPO_Mask = 0x00000000;} 

if (*(frame + FRAME_HEAD_L+3) == GPIO_HIGH)
  	XGpio_DiscreteWrite(&GpioOutput,1, (GPO_Mask));
else{
	GPO_Mask = ~GPO_SEL & GPO_Mask;
    XGpio_DiscreteWrite(&GpioOutput,1, (GPO_Mask));}
      			    
```